# Demographic Analysis

# imports

In [10]:
install.packages("ggridges")
install.packages("reshape2")
install.packages("tidyverse")
install.packages("magrittr")
install.packages("broom")
install.packages("FactoMineR")
install.packages("emmeans")

library(tidyverse)
library(magrittr)
library(broom)
library(FactoMineR)
library(emmeans)
library(ggridges)
library(reshape2)



The downloaded binary packages are in
	/var/folders/tt/j3zywfxn4s364d1pj12scs_r0000gq/T//Rtmpah7fVo/downloaded_packages

The downloaded binary packages are in
	/var/folders/tt/j3zywfxn4s364d1pj12scs_r0000gq/T//Rtmpah7fVo/downloaded_packages

The downloaded binary packages are in
	/var/folders/tt/j3zywfxn4s364d1pj12scs_r0000gq/T//Rtmpah7fVo/downloaded_packages

The downloaded binary packages are in
	/var/folders/tt/j3zywfxn4s364d1pj12scs_r0000gq/T//Rtmpah7fVo/downloaded_packages

The downloaded binary packages are in
	/var/folders/tt/j3zywfxn4s364d1pj12scs_r0000gq/T//Rtmpah7fVo/downloaded_packages

The downloaded binary packages are in
	/var/folders/tt/j3zywfxn4s364d1pj12scs_r0000gq/T//Rtmpah7fVo/downloaded_packages


also installing the dependencies ‘estimability’, ‘numDeriv’, ‘mvtnorm’





The downloaded binary packages are in
	/var/folders/tt/j3zywfxn4s364d1pj12scs_r0000gq/T//Rtmpah7fVo/downloaded_packages


Welcome to emmeans.
NOTE -- Important change from versions <= 1.41:
    Indicator predictors are now treated as 2-level factors by default.
    To revert to old behavior, use emm_options(cov.keep = character(0))


Attaching package: ‘reshape2’


The following object is masked from ‘package:tidyr’:

    smiths




## load data, scripts
pca_labels.rda is old. You could update it with the new labels in place if you want using the structure of pca_labels.rda. I filled in the table in google sheets, then exported it as a csv, loaded it into R, and then resaved it as a RDA file.

In [11]:
load('./data/merged_artsengagement.rda')
load('./data/tidy_questions_best.Rda')
source('./scripts/select_helpers.R')
load('./data/pca_labels.rda')

## Demo Variables

In [ ]:
# all factors vvv
# df[df %>% sapply(is.factor) %>% as.vector, rownums]
motiv_demos <- df %>% select(contains('sr_motivation')) %>% names
identity_demos <- df %>% select(contains('sr_identity')) %>% names
participation_demos <- df %>% select(all_arts('participation')) %>% select(starts_with('sr')) %>% names
real_demos <- c('ethnic_group', 'sex', 'school', 'parented', 'income', 'artsincollege', 'hstype', 'hssize',
           'hslocation', 'hs_arts_freq', 'hs_encouragement', 'hs_required', 'hs_fees',
           'so_childhood1', 'so_childhood3', 'so_childhood5', 'sr_participated',
               'sr_highestdegreeplanned')
demo_groups <- c('real_demos','motiv_demos','identity_demos','participation_demos')

## Functions

In [ ]:
stripnames <- function(data, question) {
    data <- data %>% select('rownums', starts_with(question))
    varnames <- names(data)
    newnames <- varnames
    for (i in seq(3, length(varnames))) {
        newnames[i] <- substr(varnames[i], gregexpr("\\.\\.", varnames[i])[[1]][1]+2, nchar(varnames[i]))
    }
    names(data) <- newnames
    return(data)
}

stripVars <- function(data) {
    #remove irrelavant variables
    removenames <- names(data %>% select(matches("AIC..Words|AIC..IC_Differentiation|AIC..IC_Integration|AIC..DIAL_Differentiation|AIC..DIAL_Integration|AIC..ELAB_Differentiation|AIC..ELAB_Integration|docuscope..OralCues|docuscope..DialogCues|docuscope..XWordTokens|docuscope..XPunctuationTokens|docuscope..XTokens|LIWC..WC|LIWC..Sixltr|LIWC..Dic|LIWC..WPS|LIWC..swear|LIWC..netspeak|LIWC..assent|LIWC..nonflu|LIWC..filler|LIWC..AllPunc|LIWC..Period|LIWC..Comma|LIWC..Colon|LIWC..SemiC|LIWC..QMark|LIWC..Exclam|LIWC..Dash|LIWC..Quote|LIWC..Apostro|LIWC..Parenth|LIWC..OtherP")))
    data <- data[ , !names(data) %in% removenames]
    return(data)
}

mergeQuestion <- function(data, question) {
    if (substr(question, 3,3) == '_') {
        question <- substr(question, 4, nchar(question))
    }
    # if you're aware of other prefixes, add them here
    yrs <- c(NA,'fl','f1','f2','so','jr','sr','sl')
    data$rownums <- seq(nrow(data)) # add rownums so resulting data can be re-integrated
    mergeddf <- data.frame()
    for (yr in yrs) {
        if (question == 'definition' && is.na(yr))
            qyr <- question
        else if (is.na(yr))
            next
        else {
            qyr <- paste0(yr, '_', question)
        }
        if (data %>% select(starts_with(qyr)) %>% ncol == 0) { next }
        tempdf <- stripnames(data, qyr)
        names(tempdf)[2] <- question
        mergeddf <- bind_rows(mergeddf, tempdf)        
    }
    mergeddf <- mergeddf[complete.cases(mergeddf),]
    mergeddf <- mergeddf %>% stripVars
    rownames(mergeddf) <- c()
    return(mergeddf)
}

## alt temp

In [ ]:
temp <- demodfpcs %>% select(-matches(paste0('PC[',paste0(seq(5)[seq(5)!=p], collapse = ''),']')))
names(temp)[ncol(temp)] <- 'pc'

## all loop

In [ ]:
sig_contrasts.demos <- c()
sig_aov_posthoc <- c()
dir.create('./results', showWarnings = F)
for(question in question_names[-c(4:6,9:14)]) {
    # directories
    dir.create(paste('./results', question, sep = '/'), showWarnings = F)
    
    # data subsetting, PC
    data_subset <- mergeQuestion(df, question)
    data_subset_rows <- data_subset[[1]]
    data_subset <- data_subset[-1]
    data_subset <- data_subset[-1]
    percent <- 0.4
    data_subset <- data_subset[ lapply( data_subset, function(x) sum(x==0) / length(x) ) < percent ]
    pd <- prcomp(data_subset, retx= TRUE, center=TRUE, scale=TRUE)

    # merge with demographic cols
    pcs <- data.frame(rownum = data_subset_rows, pd$x[,seq(5)] %>% as.data.frame)
    rownums <- pcs$rownum %>% sort
    pcs <- pcs %>% arrange(rownum)
    pcs <- pcs[-1]
    demodfpcs <- bind_cols(df[rownums,c('key',real_demos)], pcs)

    # weighting by # of responses per respondent
    resps_by_key <- demodfpcs$key %>% table %>% as.data.frame
    names(resps_by_key) <- c('key','n')
    resps_by_key %<>% filter(n!=0)
    for(r in seq(nrow(demodfpcs))) {
        nresps <- resps_by_key %>% filter(key == demodfpcs[r,]$key) %>% .$n
        demodfpcs[r, names(select(demodfpcs,matches('PC[12345]')))] <- (1/nresps) *
                                                            (demodfpcs[r, names(select(demodfpcs,matches('PC[12345]')))])
    }
    demodfpcs <- demodfpcs %>% select(-'key')
    demodf <- demodfpcs %>% select(-matches('PC[12345]'))
    pcs <- demodfpcs %>% select(matches('PC[12345]'))

    # analysis
    for(p in seq(pcs)) {
        # directories
        dir.create(paste('./results', question, names(pcs)[p], sep = '/'), showWarnings = F)
        dir.create(paste('./results', question, names(pcs)[p], 'emmeans', sep = '/'), showWarnings = F)
        dir.create(paste('./results', question, names(pcs)[p], 'anova_tukey', sep = '/'), showWarnings = F)
        for(d in seq(demodf)) {
            temp <- bind_cols(pcs[p], demodf[d])
            names(temp) <- c('pc','demo')
            if((temp$demo %>% table %>% as.data.frame %>% filter(Freq != 0) %>% nrow) == 1) next

            # emmeans contrasts
            lm.out <- lm(pc ~ demo, data=temp, na.action = na.exclude)
            em.out <- emmeans(lm.out, ~ demo, data=temp, weights = 'proportional')
            contrast.out <- contrast(em.out, method="del.eff") %>% tidy

            contrast.out$p.value %<>% p.adjust(method='holm')
            contrast.out %<>% mutate(question_name=question,
                                        PC_num = paste0('PC',p), 
                                        PC_pos = pca_labels %>% 
                                                  filter(question_name==question) %>% 
                                                  select(matches(paste0(p,'_pos'))) %>%
                                                  pull %>% as.character,
                                        PC_neg = pca_labels %>% 
                                                  filter(question_name==question) %>% 
                                                  select(matches(paste0(p,'_neg'))) %>%
                                                  pull %>% as.character,
                                        demographic = names(demodf)[d]
                                     )            

            # anova tukey
            aov.out <- aov(pc ~ demo, data=temp) %>% TukeyHSD %>% tidy
            aov.out %<>% select(-term) %>% mutate(question_name=question,
                                        PC_num = paste0('PC',p), 
                                        PC_pos = pca_labels %>% 
                                                  filter(question_name==question) %>% 
                                                  select(matches(paste0(p,'_pos'))) %>%
                                                  pull %>% as.character,
                                        PC_neg = pca_labels %>% 
                                                  filter(question_name==question) %>% 
                                                  select(matches(paste0(p,'_neg'))) %>%
                                                  pull %>% as.character,
                                        demographic = names(demodf)[d]
                                     )
            
            # bind rows
            options(warn=-1)
            sig_contrasts.demos %<>% bind_rows(contrast.out)
            sig_aov_posthoc %<>% bind_rows(aov.out)
            options(warn=0)
            
            # write csvs
            write.csv(contrast.out, paste('./results',question,names(pcs)[p],'emmeans',paste0(names(demodf)[d],'.csv'),sep='/'))
            write.csv(aov.out, paste('./results',question,names(pcs)[p],'anova_tukey',paste0(names(demodf)[d],'.csv'),sep='/'))
        }
    }
}
# # write rdas
save(sig_contrasts.demos, file='./results/emmeans_contrasts.rda')
save(sig_aov_posthoc, file='./results/anova_tukey.rda')

## begin tidy graphs 

In [12]:
real_demos <- c('ethnic_group', 'sex', 'school', 'parented', 'income', 'artsincollege', 'hstype', 'hssize',
           'hslocation', 'hs_arts_freq', 'hs_encouragement', 'hs_required', 'hs_fees',
           'so_childhood1', 'so_childhood3', 'so_childhood5', 'sr_participated',
               'sr_highestdegreeplanned')

## define helper functions

In [13]:
stripnames <- function(data, question) {
    data <- data %>% select('rownums', starts_with(question))
    varnames <- names(data)
    newnames <- varnames
    for (i in seq(3, length(varnames))) {
        newnames[i] <- substr(varnames[i], gregexpr("\\.\\.", varnames[i])[[1]][1]+2, nchar(varnames[i]))
    }
    names(data) <- newnames
    return(data)
}

stripVars <- function(data) {
    #remove irrelavant variables
    removenames <- names(data %>% select(matches("AIC..Words|AIC..IC_Differentiation|AIC..IC_Integration|AIC..DIAL_Differentiation|AIC..DIAL_Integration|AIC..ELAB_Differentiation|AIC..ELAB_Integration|docuscope..OralCues|docuscope..DialogCues|docuscope..XWordTokens|docuscope..XPunctuationTokens|docuscope..XTokens|LIWC..WC|LIWC..Sixltr|LIWC..Dic|LIWC..WPS|LIWC..swear|LIWC..netspeak|LIWC..assent|LIWC..nonflu|LIWC..filler|LIWC..AllPunc|LIWC..Period|LIWC..Comma|LIWC..Colon|LIWC..SemiC|LIWC..QMark|LIWC..Exclam|LIWC..Dash|LIWC..Quote|LIWC..Apostro|LIWC..Parenth|LIWC..OtherP")))
    data <- data[ , !names(data) %in% removenames]
    return(data)
}

mergeQuestion <- function(data, question) {
    if (substr(question, 3,3) == '_') {
        question <- substr(question, 4, nchar(question))
    }
    # if you're aware of other prefixes, add them here
    yrs <- c(NA,'fl','f1','f2','so','jr','sr','sl')
    data$rownums <- seq(nrow(data)) # add rownums so resulting data can be re-integrated
    mergeddf <- data.frame()
    for (yr in yrs) {
        if (question == 'definition' && is.na(yr))
            qyr <- question
        else if (is.na(yr))
            next
        else {
            qyr <- paste0(yr, '_', question)
        }
        if (data %>% select(starts_with(qyr)) %>% ncol == 0) { next }
        tempdf <- stripnames(data, qyr)
        names(tempdf)[2] <- question
        mergeddf <- bind_rows(mergeddf, tempdf)        
    }
    mergeddf <- mergeddf[complete.cases(mergeddf),]
    mergeddf <- mergeddf %>% stripVars
    rownames(mergeddf) <- c()
    return(mergeddf)
}

In [14]:
question <- question_names[-c(4:6,9:14)][3]

In [15]:
# data subsetting, PC
data_subset <- mergeQuestion(df, question)
data_subset_rows <- data_subset[[1]]
data_subset <- data_subset[-1]
data_subset <- data_subset[-1]
percent <- 0.4
data_subset <- data_subset[ lapply( data_subset, function(x) sum(x==0) / length(x) ) < percent ]
pd <- prcomp(data_subset, retx= TRUE, center=TRUE, scale=TRUE)

# merge with demographic cols
pcs <- data.frame(rownum = data_subset_rows, pd$x[,seq(5)] %>% as.data.frame)
rownums <- pcs$rownum %>% sort
pcs <- pcs %>% arrange(rownum)
pcs <- pcs[-1]
demodfpcs <- bind_cols(df[rownums,c('key',real_demos)], pcs)

# weighting by # of responses per respondent
resps_by_key <- demodfpcs$key %>% table %>% as.data.frame
names(resps_by_key) <- c('key','n')
resps_by_key %<>% filter(n!=0)
for(r in seq(nrow(demodfpcs))) {
    nresps <- resps_by_key %>% filter(key == demodfpcs[r,]$key) %>% .$n
    demodfpcs[r, names(select(demodfpcs,matches('PC[12345]')))] <- (1/nresps) *
                                                        (demodfpcs[r, names(select(demodfpcs,matches('PC[12345]')))])
}
demodfpcs <- demodfpcs %>% select(-'key')
demodf <- demodfpcs %>% select(-matches('PC[12345]'))
pcs <- demodfpcs %>% select(matches('PC[12345]'))

# analysis

### Filter demographics to those within min & max levels (inclusive) into small_demos variable

In [16]:
min = 2
max = 24
temp <- demodfpcs %>% sapply(function(x)(x %>% levels %>% length)) %>% as.data.frame
temp$demo <- rownames(temp)
temp <- temp %>% filter(. >= min & max >= .)
small_demos <- temp %>% pull(demo)
small_demos

[1] "ethnic_group"            "sex"                    
 [3] "school"                  "parented"               
 [5] "income"                  "artsincollege"          
 [7] "hstype"                  "hssize"                 
 [9] "hslocation"              "hs_arts_freq"           
[11] "hs_encouragement"        "hs_required"            
[13] "hs_fees"                 "so_childhood1"          
[15] "so_childhood3"           "so_childhood5"          
[17] "sr_participated"         "sr_highestdegreeplanned"

## Loop over all questions, generating PCs, munging their data, and finally generating graphs for them all
I recommend letting the code run for say 10 seconds then hit i twice to stop it to then print out / explore the variables being used within the ggplot code.

In [ ]:
dir_to_save_results = './results-jan'
plot_type = 'violin_plots' # plural
output_format = '.pdf' # with dot

dir.create(dir_to_save_results, showWarnings = F)
# for all stm'd questions
for(question in question_names[-c(4:6,9:14)]) {
    dir.create(paste(dir_to_save_results, question, sep = '/'), showWarnings = F)
    
    # data subsetting, PC creation
    data_subset <- mergeQuestion(df, question)
    data_subset_rows <- data_subset[[1]]
    data_subset <- data_subset[-1]
    data_subset <- data_subset[-1]
    percent <- 0.4
    data_subset <- data_subset[ lapply( data_subset, function(x) sum(x==0) / length(x) ) < percent ]
    pd <- prcomp(data_subset, retx= TRUE, center=TRUE, scale=TRUE)

    # merge PCs with demographic cols
    pcs <- data.frame(rownum = data_subset_rows, pd$x[,seq(5)] %>% as.data.frame)
    rownums <- pcs$rownum %>% sort
    pcs <- pcs %>% arrange(rownum)
    pcs <- pcs[-1]
    demodfpcs <- bind_cols(df[rownums,c('key',real_demos)], pcs)

    # weighting by # of responses per respondent
    resps_by_key <- demodfpcs$key %>% table %>% as.data.frame
    names(resps_by_key) <- c('key','n')
    resps_by_key %<>% filter(n!=0)
    for(r in seq(nrow(demodfpcs))) {
        nresps <- resps_by_key %>% filter(key == demodfpcs[r,]$key) %>% .$n
        demodfpcs[r, names(select(demodfpcs,matches('PC[12345]')))] <- (1/nresps) *
                                                        (demodfpcs[r, names(select(demodfpcs,matches('PC[12345]')))])
    }
                                       
    # separate PCs and demographics
    demodfpcs <- demodfpcs %>% select(-'key')
    demodf <- demodfpcs %>% select(-matches('PC[12345]'))
    pcs <- demodfpcs %>% select(matches('PC[12345]'))

                  
    # plotting (by PC)
    for(p in seq(pcs)) {
        # combine individual PC with demographics
        temp <- bind_cols(pcs[p], demodf)
        names(temp) <- c("pc", names(temp)[-1])
        
        
        # for each demographic...
        for(d in seq(small_demos)) {
            mu <- temp %>% na.omit %>% group_by_at(vars(small_demos[d])) %>% 
                summarise(grp.median=median(pc), grp.mean=mean(pc), grp.sd=sd(pc))
#density plot
               ggplot(na.omit(demodfpcs), aes_string(x=names(pcs)[p], fill=small_demos[d], color=small_demos[d])) +
              #  geom_density_ridges(position='identity', alpha=.2) +
               geom_density(position='identity', alpha=.2) +
                
   #violin_plot         
            #g <- ggplot(na.omit(demodfpcs), aes_string(x=names(pcs)[p], y=small_demos[d]))
            #g + geom_violin(scale="area") + 

                xlim(round(na.omit(pcs[[p]]) %>% summary %>% tidy %>% .$minimum)-0.5, 
                     round(na.omit(pcs[[p]]) %>% summary %>% tidy %>% .$maximum)+.5) +
                geom_vline(data=mu, aes_string(xintercept='grp.median', color=names(mu)[1]),
                         linetype="dashed") +
                labs(subtitle=paste0('pos-x: ', pca_labels %>% 
                      filter(question_name==question) %>% 
                      select(matches(paste0(p,'_pos'))) %>%
                      pull %>% as.character,
                     '\nneg-x: ', pca_labels %>% 
                      filter(question_name==question) %>% 
                      select(matches(paste0(p,'_neg'))) %>%
                      pull %>% as.character)) +
                theme(plot.subtitle = element_text(size=10))

            # saving
            dir.create(paste(dir_to_save_results, question, names(pcs)[p], sep='/'), showWarnings = F)
            dir.create(paste(dir_to_save_results, question, names(pcs)[p], plot_type,sep='/'), showWarnings = F)
            ggsave(paste(dir_to_save_results, question, names(pcs)[p], plot_type, 
                         paste0(small_demos[d], output_format),sep='/'), 
                   width = 6, height=5)
        }
    }
}

Warning message:
“Groups with fewer than two data points have been dropped.”
Warning message:
“Groups with fewer than two data points have been dropped.”
Warning message:
“Groups with fewer than two data points have been dropped.”
Warning message:
“Groups with fewer than two data points have been dropped.”
Warning message:
“Groups with fewer than two data points have been dropped.”
Warning message:
“Groups with fewer than two data points have been dropped.”
Warning message:
“Groups with fewer than two data points have been dropped.”
Warning message:
“Groups with fewer than two data points have been dropped.”
Warning message:
“Groups with fewer than two data points have been dropped.”
Warning message:
“Groups with fewer than two data points have been dropped.”
Warning message:
“Groups with fewer than two data points have been dropped.”
Warning message:
“Groups with fewer than two data points have been dropped.”
Warning message:
“Groups with fewer than two data points have been dropped.”

In [95]:
names(pcs)[p]

[1] "PC5"

In [35]:
ggplot(summaryValues,aes(factor(Discipline), y = pointEst , ymin = minCI, ymax = maxCI,
          color=factor(Discipline), shape=factor(SigEff, labels = c("Not Sig.", "p <= .05")))) +
          geom_pointrange() + facet_wrap(~Topic, shrink = TRUE, as.table = TRUE, labeller = topic_labels_panel) + 
          theme(legend.justification = "top", axis.title.x=element_blank(), axis.text.x=element_blank(), axis.ticks.x=element_blank()) + 
           scale_colour_manual(name="Primary Discipline", values=a2ru) + scale_shape_manual(name="Significant Difference?", values = c(19,8)) + labs(y = "Proportion") +
           labs(title = "Topical Prevalence Plot across Different Topics") 
    ggsave(paste(folder_id, q_id, "TopicPrevByTopic.pdf", sep = ""), width = width, height = height, useDingbats=FALSE)

ERROR: Error in ggplot(summaryValues, aes(factor(Discipline), y = pointEst, ymin = minCI, : object 'summaryValues' not found


In [85]:
small_demos

[1] "ethnic_group"            "sex"                    
 [3] "school"                  "parented"               
 [5] "income"                  "artsincollege"          
 [7] "hstype"                  "hssize"                 
 [9] "hslocation"              "hs_arts_freq"           
[11] "hs_encouragement"        "hs_required"            
[13] "hs_fees"                 "so_childhood1"          
[15] "so_childhood3"           "so_childhood5"          
[17] "sr_participated"         "sr_highestdegreeplanned"

In [108]:
    plotdata <- melt(na.omit(demodfpcs))


Using key, ethnic_group, sex, school, parented, income, artsincollege, hstype, hssize, hslocation, hs_arts_freq, hs_encouragement, hs_required, hs_fees, so_childhood1, so_childhood3, so_childhood5, sr_participated, sr_highestdegreeplanned as id variables


In [113]:
reshapedata <- melt(pcs)


No id variables; using all as measure variables


In [120]:
names(temp)

[1] "pc"                      "ethnic_group"           
 [3] "sex"                     "school"                 
 [5] "parented"                "income"                 
 [7] "artsincollege"           "hstype"                 
 [9] "hssize"                  "hslocation"             
[11] "hs_arts_freq"            "hs_encouragement"       
[13] "hs_required"             "hs_fees"                
[15] "so_childhood1"           "so_childhood3"          
[17] "so_childhood5"           "sr_participated"        
[19] "sr_highestdegreeplanned"